In [1]:
import model_lib
import numpy as np
import warnings
warnings.filterwarnings('ignore', '.*output shape of zoom.*')
import pickle
import importlib
importlib.reload(model_lib)
import os
import time

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [2]:
# config to train
# TODO: check Config is correct
class ProposalConfig():
    NAME = "InSegm"
    GPU_COUNT = 1
    # online training
    IMAGES_PER_GPU = 1
    STEPS_PER_EPOCH = 100
    NUM_WORKERS = 1
    PIN_MEMORY = True
    DATA_ORDER = "cw_ins"
    VALIDATION_STEPS = 20
    # including gt
    NUM_CLASSES = 81
    
    # only flips
    MEAN_PIXEL = np.array([0.485, 0.456, 0.406],dtype=np.float32).reshape(1,1,-1)
    STD_PIXEL = np.array([0.229, 0.224, 0.225],dtype=np.float32).reshape(1,1,-1)
    CLASS_NAMES = [
        'BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
        'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
        'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
        'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
        'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
        'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
        'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
        'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
        'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
        'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
        'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
        'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
        'scissors', 'teddy bear', 'hair drier', 'toothbrush'
    ]
    WIDTH = 224
    HEIGHT = 224
    CROP_SIZE = 224
    def __init__(self):
        self.BATCH_SIZE = self.IMAGES_PER_GPU * self.GPU_COUNT
        self.IMAGE_SHAPE = (self.WIDTH, self.HEIGHT,3)

    def display(self):
        """Display Configuration values."""
        print("\nConfigurations:")
        for a in dir(self):
            if not a.startswith("__") and not callable(getattr(self, a)):
                print("{:30} {}".format(a, getattr(self, a)))
        print("\n")

In [3]:
train_image_dir = "/media/Data1/interns/aravind/train2017/"
# train_image_dir = "/media/Data1/interns/aravind/val2017/"
val_image_dir = "/media/Data1/interns/aravind/val2017/"
config = ProposalConfig()
model_dir = "./models/"
train_pickle = "/media/Data1/interns/aravind/re/data/train_cwid.pickle"
# train_pickle = "/media/Data1/interns/aravind/re/data/val_cwid.pickle"
val_pickle = "/media/Data1/interns/aravind/re/data/val_cwid.pickle"

In [4]:
# with open(train_pickle,"rb") as train_ann:
#     train_cwid = pickle.load(train_ann)
with open(val_pickle,"rb") as val_ann:
    val_cwid = pickle.load(val_ann)

In [5]:
# train_loader = model_lib.get_loader(train_cwid,config,train_image_dir)
val_loader = model_lib.get_loader(val_cwid,config,val_image_dir)

In [ ]:
import torch
import torch.nn.functional as F
from PIL import Image
net = model_lib.MultiHGModel()

pretrained_dict = torch.load(model_dir+"hg3.pt")
net_dict = net.state_dict()

pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in net_dict}
net_dict.update(pretrained_dict) 
net.load_state_dict(net_dict)

# net.vgg.load_state_dict(torch.load(model_dir+"model_vgg_class_only.pt").vgg)
# net.classifier.load_state_dict(torch.load(model_dir+"model_vgg_class_only.pt").classifier)
net = net.cuda()
with torch.no_grad():
    for i,data in enumerate(val_loader):
        batch_images,batch_impulses,batch_gt_responses,batch_bboxes,batch_one_hot = data
        print(config.CLASS_NAMES[np.argmax(batch_one_hot.numpy())])
        a,b,c,d= batch_images.numpy(), batch_impulses.numpy(),batch_gt_responses.numpy(),batch_bboxes.numpy()
        a = np.moveaxis(a,1,-1)
        b = np.moveaxis(b,1,-1)
        c = np.moveaxis(c,1,-1)
        d = np.moveaxis(d,1,-1)
        Image.fromarray(((a[0]*config.STD_PIXEL + config.MEAN_PIXEL)*255).astype(np.uint8),"RGB").show()
        Image.fromarray((b[0][:,:,0]*128).astype(np.uint8),"L").show()
        Image.fromarray((c[0][:,:,0]*128).astype(np.uint8),"L").show()
        Image.fromarray((d[0][:,:,0]*128).astype(np.uint8),"L").show()
        batch_images,batch_impulses,batch_gt_responses,batch_bboxes,batch_one_hot = batch_images.cuda(),batch_impulses.cuda(),batch_gt_responses.cuda(),batch_bboxes.cuda(),batch_one_hot.cuda()
        pred_class,pred_masks = net([batch_images,batch_impulses])
        pred_class = F.softmax(pred_class,dim=-1).squeeze()
        maxs, indices = torch.topk(pred_class,5,-1)
        pred_mask = pred_masks[1].squeeze()
        pred_mask = F.sigmoid(pred_mask)
#         pred_mask = F.threshold(pred_mask,0.9,0)
#         pred_mask = (pred_mask>0).float()
        pred_mask = pred_mask.squeeze().cpu().numpy()*255
        Image.fromarray(pred_mask.astype(np.uint8),"L").convert("RGB").show()
        pred_mask = pred_masks[0].squeeze()
        pred_mask = F.sigmoid(pred_mask)
#         pred_mask = F.threshold(pred_mask,0.9,0)
#         pred_mask = (pred_mask>0).float()
        pred_mask = pred_mask.squeeze().cpu().numpy()*255
        Image.fromarray(pred_mask.astype(np.uint8),"L").convert("RGB").show()

        for i in range(5):
            print(maxs[i],indices[i],config.CLASS_NAMES[int(indices[i])])
        input()

wine glass
tensor(0.5195, device='cuda:0') tensor(41, device='cuda:0') wine glass
tensor(0.2934, device='cuda:0') tensor(42, device='cuda:0') cup
tensor(0.1081, device='cuda:0') tensor(40, device='cuda:0') bottle
tensor(1.00000e-02 *
       5.1903, device='cuda:0') tensor(61, device='cuda:0') dining table
tensor(1.00000e-02 *
       1.8489, device='cuda:0') tensor(0, device='cuda:0') BG

bowl
tensor(0.3857, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(0.1797, device='cuda:0') tensor(2, device='cuda:0') bicycle
tensor(0.1597, device='cuda:0') tensor(26, device='cuda:0') umbrella
tensor(0.1022, device='cuda:0') tensor(25, device='cuda:0') backpack
tensor(1.00000e-02 *
       9.2051, device='cuda:0') tensor(27, device='cuda:0') handbag

BG
tensor(0.9690, device='cuda:0') tensor(7, device='cuda:0') train
tensor(1.00000e-02 *
       1.1141, device='cuda:0') tensor(14, device='cuda:0') bench
tensor(1.00000e-03 *
       7.2045, device='cuda:0') tensor(0, device='cuda:0') BG
tenso


kite
tensor(1.0000, device='cuda:0') tensor(34, device='cuda:0') kite
tensor(1.00000e-05 *
       2.1183, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-06 *
       6.1532, device='cuda:0') tensor(15, device='cuda:0') bird
tensor(1.00000e-06 *
       3.1907, device='cuda:0') tensor(32, device='cuda:0') snowboard
tensor(1.00000e-06 *
       3.1645, device='cuda:0') tensor(5, device='cuda:0') airplane

sports ball
tensor(0.5035, device='cuda:0') tensor(33, device='cuda:0') sports ball
tensor(0.2374, device='cuda:0') tensor(1, device='cuda:0') person
tensor(0.1837, device='cuda:0') tensor(75, device='cuda:0') clock
tensor(1.00000e-02 *
       2.2325, device='cuda:0') tensor(68, device='cuda:0') cell phone
tensor(1.00000e-02 *
       1.3039, device='cuda:0') tensor(57, device='cuda:0') chair

dog
tensor(0.9999, device='cuda:0') tensor(17, device='cuda:0') dog
tensor(1.00000e-05 *
       1.4824, device='cuda:0') tensor(18, device='cuda:0') horse
tensor(1.00000e-05 *
   


person
tensor(1., device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-09 *
       1.8777, device='cuda:0') tensor(0, device='cuda:0') BG
tensor(1.00000e-09 *
       1.5355, device='cuda:0') tensor(28, device='cuda:0') tie
tensor(1.00000e-09 *
       1.0412, device='cuda:0') tensor(27, device='cuda:0') handbag
tensor(1.00000e-10 *
       9.6025, device='cuda:0') tensor(35, device='cuda:0') baseball bat

person
tensor(0.9996, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-04 *
       1.3714, device='cuda:0') tensor(78, device='cuda:0') teddy bear
tensor(1.00000e-05 *
       5.2915, device='cuda:0') tensor(68, device='cuda:0') cell phone
tensor(1.00000e-05 *
       4.6721, device='cuda:0') tensor(28, device='cuda:0') tie
tensor(1.00000e-05 *
       2.6221, device='cuda:0') tensor(17, device='cuda:0') dog

bottle
tensor(0.6118, device='cuda:0') tensor(27, device='cuda:0') handbag
tensor(0.3021, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.


person
tensor(0.8923, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-02 *
       4.4448, device='cuda:0') tensor(17, device='cuda:0') dog
tensor(1.00000e-02 *
       2.1394, device='cuda:0') tensor(15, device='cuda:0') bird
tensor(1.00000e-02 *
       1.5194, device='cuda:0') tensor(19, device='cuda:0') sheep
tensor(1.00000e-03 *
       6.7001, device='cuda:0') tensor(18, device='cuda:0') horse

bird
tensor(1., device='cuda:0') tensor(15, device='cuda:0') bird
tensor(1.00000e-10 *
       9.4758, device='cuda:0') tensor(20, device='cuda:0') cow
tensor(1.00000e-10 *
       5.2891, device='cuda:0') tensor(18, device='cuda:0') horse
tensor(1.00000e-10 *
       5.0761, device='cuda:0') tensor(19, device='cuda:0') sheep
tensor(1.00000e-10 *
       1.1929, device='cuda:0') tensor(45, device='cuda:0') spoon

cup
tensor(0.8039, device='cuda:0') tensor(42, device='cuda:0') cup
tensor(1.00000e-02 *
       5.8310, device='cuda:0') tensor(40, device='cuda:0') bottle
tensor(1.00


chair
tensor(0.8852, device='cuda:0') tensor(27, device='cuda:0') handbag
tensor(1.00000e-02 *
       4.3694, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(1.00000e-02 *
       1.5048, device='cuda:0') tensor(37, device='cuda:0') skateboard
tensor(1.00000e-03 *
       6.9305, device='cuda:0') tensor(35, device='cuda:0') baseball bat
tensor(1.00000e-03 *
       6.6430, device='cuda:0') tensor(29, device='cuda:0') suitcase

keyboard
tensor(0.9789, device='cuda:0') tensor(67, device='cuda:0') keyboard
tensor(1.00000e-02 *
       1.8555, device='cuda:0') tensor(64, device='cuda:0') laptop
tensor(1.00000e-03 *
       1.5614, device='cuda:0') tensor(68, device='cuda:0') cell phone
tensor(1.00000e-04 *
       4.4879, device='cuda:0') tensor(66, device='cuda:0') remote
tensor(1.00000e-04 *
       2.6387, device='cuda:0') tensor(74, device='cuda:0') book

potted plant
tensor(0.6002, device='cuda:0') tensor(59, device='cuda:0') potted plant
tensor(0.3076, device='cuda:0') tensor(0, 